In [3]:
import search.unionable_table_search as uts
import search.joinable_table_search as jts
import post_processing.filtering_reranking as fr
import pickle

![pipeline](pipeline_illustration.svg)

## Unionable Table Search 
Once the data lake embeddings have been computed following the instructions described in [here](embedding_computation/README.md), we can execute unionable table search. <br>
In this demo, we have already computed embeddings for SANTOS data lake provided with SANTOS benchmark with the following configurations [#0](embedding_computation/experiments.md). 

### Using Faiss 

In [7]:
datalake_embeddings = './output/hytrel_embedding/santos/hytrel_datalake_columns_0.pkl'
query_embeddings = './output/hytrel_embedding/santos/hytrel_query_columns_0.pkl'
k = 10
## select a query dataset
with open(query_embeddings, 'rb') as f:
    query_columns_hytrel = pickle.load(f)
with open(datalake_embeddings, 'rb') as f:
    datalake_columns_hytrel = pickle.load(f)
query_table = query_columns_hytrel[0] ## corresponding to 'cihr_co-applicant_b.csv'
candidates, build_duration, query_duration = uts.approximate_unionable_dataset_search([query_columns_hytrel[0]], datalake_columns_hytrel,k,compress_method='max')
candidates = candidates['cihr_co-applicant_b.csv']


#### Candidates 

In [8]:
for i in range(len(candidates)):
    print(f'candidate in {i+1} place: {candidates[i]}')

print(f'Index build duration: {build_duration}')
print(f'Query duration: {query_duration}')

candidate in 1 place: cihr_co-applicant_b.csv
candidate in 2 place: cihr_co-applicant_9.csv
candidate in 3 place: cihr_co-applicant_7.csv
candidate in 4 place: cihr_co-applicant_6.csv
candidate in 5 place: cihr_co-applicant_8.csv
candidate in 6 place: cihr_co-applicant_3.csv
candidate in 7 place: cihr_co-applicant_5.csv
candidate in 8 place: cihr_co-applicant_1.csv
candidate in 9 place: cihr_co-applicant_0.csv
candidate in 10 place: cihr_co-applicant_2.csv
Index build duration: 0.0013661384582519531
Query duration: 8.20159912109375e-05


### Clustering-based search 
Once the embeddings have been computed. We can preform hierarchal clustering on the computed data lake embeddings. Instructions to compute heirarchal clustering can be found [here](clustering/) <br>
In this demo, we have ran clustering on column embeddings with configuration [0](/embedding_computation/experiments.md) and cluster count of [811](clustering/experiments.md)


In [9]:
clustering = './output/clustering/santos/clustering_811_santos_run_id_0.pkl'
with open(clustering, 'rb') as f:
    clustering_result = pickle.load(f)
datalake = list(set(clustering_result['dataset'].unique()))
query = 'cihr_co-applicant_b.csv'
k = 10
res = uts.unionable_table_search_using_clustering([query], datalake, clustering_result,k)
candidates = res[query]

TypeError: Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)

#### Candidates

In [10]:
for i in range(len(candidates)):
    print(f'candidate in {i+1} place: {candidates[i]}')

candidate in 1 place: cihr_co-applicant_b.csv
candidate in 2 place: cihr_co-applicant_9.csv
candidate in 3 place: cihr_co-applicant_7.csv
candidate in 4 place: cihr_co-applicant_6.csv
candidate in 5 place: cihr_co-applicant_8.csv
candidate in 6 place: cihr_co-applicant_3.csv
candidate in 7 place: cihr_co-applicant_5.csv
candidate in 8 place: cihr_co-applicant_1.csv
candidate in 9 place: cihr_co-applicant_0.csv
candidate in 10 place: cihr_co-applicant_2.csv


## Joinable Table Search
Once the data lake embeddings have been computed following the instructions described in [here](embedding_computation/README.md), we can execute unionable table search. <br>
In this demo, we have already computed embeddings for NextiaJD testbedS data lake provided with NextiaJD benchmark with the following configurations [#4](embedding_computation/experiments.md). 

### Using Faiss

In [11]:
datalake_embeddings = './output/hytrel_embedding/nextiajd/testbedS/hytrel_datalake_columns_4.pkl'
query_embeddings = './output/hytrel_embedding/nextiajd/testbedS/hytrel_query_columns_4.pkl'
with open(query_embeddings, 'rb') as f:
    query_columns_hytrel = pickle.load(f)
with open(datalake_embeddings, 'rb') as f:
    datalake_columns_hytrel = pickle.load(f)

res, build_duration, query_duration = jts.joinable_dataset_search([query_columns_hytrel[61]], datalake_columns_hytrel,1000,'testbedS')
candidates = res[('agri-environmental-indicators-emissions-by-sector.csv',
  'Area')]


num_datalake_columns:  2553
build regular index using faiss
number of queries: 1


#### Candidates

In [12]:
for i in range(len(candidates)):
    print(f'candidate in {i+1} place: {candidates[i]}')

candidate in 1 place: ('emissions_agriculture_energy_e_all_data_norm.csv', 'Country')
candidate in 2 place: ('global-greenhouse-gas-emissions0.csv', 'Country')
candidate in 3 place: ('global-innovation-index-2015.csv', 'COUNTRY_NAME')
candidate in 4 place: ('population-estimates-and-projections-1960-2050.csv', 'COUNTRY')
candidate in 5 place: ('global-greenhouse-gas-emissions.csv', 'Country')
candidate in 6 place: ('listings_summary.csv', 'host_name')
candidate in 7 place: ('makemytrip_com-travel_sample.csv', 'city')
candidate in 8 place: ('listings_detailed.csv', 'host_name')
candidate in 9 place: ('listings_summary.csv', 'host_neighbourhood')
candidate in 10 place: ('listings_summary.csv', 'neighbourhood')


### Post-processing: Filtering, and Reranking

In [13]:
datalake_source = '/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedS/datasets'
filtered,overlap_est = fr.run_lsh_ensemble(datalake_source, res, num_perm=256, threshold=0.5, num_part=32)


Running LSH ensemble for query ('agri-environmental-indicators-emissions-by-sector.csv', 'Area')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/alaaalmutawa/Documents/Thesis/nextiajd/testbedS/datasets/agri-environmental-indicators-emissions-by-sector.csv'

#### Filtered Candidates

In [5]:
filtered

{('agri-environmental-indicators-emissions-by-sector.csv',
  'Area'): [('emissions_agriculture_energy_e_all_data_norm.csv',
   'Country'), ('global-greenhouse-gas-emissions0.csv', 'Country'), ('global-greenhouse-gas-emissions.csv',
   'Country'), ('global-innovation-index-2015.csv', 'COUNTRY_NAME')]}

#### Reranked

In [10]:
reranked = fr.rank_table(overlap_est)
candidates_reranked = reranked[('agri-environmental-indicators-emissions-by-sector.csv',
  'Area')]
for i in range(len(candidates_reranked)):
    print(f'candidate in {i+1} place: {candidates_reranked[i]} in the original list {candidates[i]}')

candidate in 1 place: ('emissions_agriculture_energy_e_all_data_norm.csv', 'Country') in the original list ('emissions_agriculture_energy_e_all_data_norm.csv', 'Country')
candidate in 2 place: ('global-greenhouse-gas-emissions0.csv', 'Country') in the original list ('global-greenhouse-gas-emissions0.csv', 'Country')
candidate in 3 place: ('global-greenhouse-gas-emissions.csv', 'Country') in the original list ('global-innovation-index-2015.csv', 'COUNTRY_NAME')
candidate in 4 place: ('global-innovation-index-2015.csv', 'COUNTRY_NAME') in the original list ('population-estimates-and-projections-1960-2050.csv', 'COUNTRY')
candidate in 5 place: ('population-estimates-and-projections-1960-2050.csv', 'COUNTRY') in the original list ('global-greenhouse-gas-emissions.csv', 'Country')
candidate in 6 place: ('listings_summary.csv', 'host_name') in the original list ('listings_summary.csv', 'host_name')
candidate in 7 place: ('makemytrip_com-travel_sample.csv', 'city') in the original list ('make